In [2]:
import numpy as np

samples=["The cat sat on the mat.", 'The dog ate my homework.']
token_index={}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word]=len(token_index)+1
            
max_length=10
results=np.zeros(shape=(len(samples),max_length,max(token_index.values())+1))
for i,sample in enumerate(samples): #sentense 1
    for j,word in list(enumerate(sample.split()))[:max_length]:
        index=token_index.get(word)
        results[i,j,index]=1.

In [14]:
import string

characters=string.printable # 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIFKLMNOPQRSTUVWXyZ...
token_index=dict(zip(characters,range(1,len(characters)+1))) #{"0":1,'1':2 .... 'A':37 ...}
max_length=50
results=np.zeros((len(samples),max_length,max(token_index.values())+1))
for i,sample in enumerate(samples):
    for j,character in enumerate(sample):
        index=token_index.get(character)
        results[i,j,index]=1.
        

In [17]:
results.shape #sample n , token n , all token n

(2, 50, 101)

In [18]:
from keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)

Using TensorFlow backend.


In [37]:
sequences=tokenizer.texts_to_sequences(samples)
print(sequences)
one_hot_results = tokenizer.texts_to_matrix(samples,mode='binary') 
word_index=tokenizer.word_index
print(word_index,len(word_index))
#tokenizer.texts_to_matrix == .texts_to_sequences + .sequences_to_matrix

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9} 9


In [38]:
# One-Hot-Hashing
dimensionality=1000 # 단어를 크기가 1000인 벡터로 저장. 그 이상의 단어가 있다면 해싱 충돌이 발생 인코딩 정확도 감소
max_length = 10

results=np.zeros((len(samples),max_length,dimensionality))
for i,sample in enumerate(samples):
    for j,word in list(enumerate(sample.split()))[:max_length]:
        index=abs(hash(word))%dimensionality #0~1000사이의 랜덤한 정수 인덱스로 변환
        results[i,j,index]=1.

In [1]:
from keras.layers import Embedding
embedding_layer=Embedding(1000,64) #(num of token, dimension) word index ( 1~ 999)


Using TensorFlow backend.


In [2]:
from keras.datasets import imdb
from keras  import preprocessing

max_features=10000
maxlen = 20
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words = max_features) #x_train 25000 comment each [15,78,49,82,79 ...]
x_train2=preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen) #list(25000,) ->ndarray (25000,maxlen) 
x_test2=preprocessing.sequence.pad_sequences(x_test,maxlen=maxlen)

In [41]:
print(sorted(x_train[0][-20:]),'\n')
print(sorted(x_train2[0]))

[5, 12, 15, 16, 16, 16, 16, 19, 32, 32, 38, 65, 88, 103, 113, 178, 283, 1334, 4472, 5345] 

[5, 12, 15, 16, 16, 16, 16, 19, 32, 32, 38, 65, 88, 103, 113, 178, 283, 1334, 4472, 5345]


In [42]:
from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding

model=Sequential()
model.add(Embedding(10000,8,input_length=maxlen))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [43]:
history=model.fit(x_train2,y_train,epochs=10,batch_size=32,validation_split=0.2)




Train on 20000 samples, validate on 5000 samples
Epoch 1/10





20000/20000 [==============================] - 2s 110us/step - loss: 0.6759 - acc: 0.6045 - val_loss: 0.6398 - val_acc: 0.6802
Epoch 2/10
20000/20000 [==============================] - 2s 86us/step - loss: 0.5658 - acc: 0.7426 - val_loss: 0.5468 - val_acc: 0.7204
Epoch 3/10
20000/20000 [==============================] - 2s 92us/step - loss: 0.4752 - acc: 0.7805 - val_loss: 0.5114 - val_acc: 0.7376
Epoch 4/10
20000/20000 [==============================] - 2s 88us/step - loss: 0.4264 - acc: 0.8077 - val_loss: 0.5008 - val_acc: 0.7434
Epoch 5/10
20000/20000 [==============================] - 2s 89us/step - loss: 0.3932 - acc: 0.8256 - val_loss: 0.4981 - val_acc: 0.7542
Epoch 6/10
20000/20000 [==============================] - 2s 88us/step - loss: 0.3672 - acc: 0.8392 - val_loss: 0.5013 - val_acc: 0.7532
Epoch 7/10
20000/20000 [==============================] - 2s 90us/step - loss: 0.3439 - acc: 0.8534 - val_loss: 0.5051 -

In [44]:
import os

imdb_dir='../../data/aclImdb/aclImdb'
train_dir=os.path.join(imdb_dir,'train')
labels=[]
texts=[]

for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir,label_type)  #  dir /train_dir/label_type
    for fname in os.listdir(dir_name):    # get  files & dir names in dir_name
        if fname[-4:] == '.txt':
            f=open(os.path.join(dir_name,fname),encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [49]:
len(texts) , type(texts)

(25000, list)

In [63]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen=100
training_samples=200
validation_samples=10000
max_words=10000 # 빈도높은 10000개 단어

tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

word_index=tokenizer.word_index
print('%s unique tokens.' %len(word_index))

data=pad_sequences(sequences,maxlen=maxlen)
labels=np.asarray(labels) # copy=false shallow
print('Data Tensor Shape :',data.shape)
print('Labels Tensor Shape:',labels.shape)

indices=np.arange(data.shape[0]) # num of documents
np.random.shuffle(indices)
data = data[indices]
labels= labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples : training_samples+validation_samples]
y_val=labels[training_samples : training_samples+validation_samples]

88582 unique tokens.
Data Tensor Shape : (25000, 100)
Labels Tensor Shape: (25000,)


In [68]:
glove_dir = '../../data/data/glove.6B'

embedding_index={}
f=open(os.path.join(glove_dir,'glove.6B.100d.txt'),encoding='utf8')
for line in f:
    values=line.split()
    word = values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=coefs
f.close()
print('Found %s Word Vector.'%len(embedding_index))

Found 400000 Word Vector.


In [70]:
embedding_dim = 100
embedding_matrix=np.zeros((max_words,embedding_dim))
for word, i in word_index.items(): # 'the' : 1, 'and' :2
    if i<max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector

In [72]:
embedding_matrix.shape

(10000, 100)

In [74]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,Flatten

model = Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False 

In [76]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))
model.save_weights('pre_trained_glove_model.h5')

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 1s 6ms/step - loss: 1.6330 - acc: 0.5250 - val_loss: 0.6959 - val_acc: 0.5127
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.7495 - acc: 0.5800 - val_loss: 0.6921 - val_acc: 0.5247
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5256 - acc: 0.7750 - val_loss: 1.0766 - val_acc: 0.4936
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5688 - acc: 0.6800 - val_loss: 0.8138 - val_acc: 0.5305
Epoch 5/10
200/200 [==============================] - 1s 4ms/step - loss: 0.4678 - acc: 0.7250 - val_loss: 0.7276 - val_acc: 0.5355
Epoch 6/10
200/200 [==============================] - 1s 4ms/step - loss: 0.1929 - acc: 0.9700 - val_loss: 1.4134 - val_acc: 0.4938
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.1935 - acc: 0.9050 - val_loss: 0.9396 - val_acc: 0.5150
Epoch 8/10
200/200 [========

In [80]:
#OVERFITTING

model=Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [81]:
x_train.shape

(200, 100)

In [82]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 1s 7ms/step - loss: 0.6949 - acc: 0.4650 - val_loss: 0.6955 - val_acc: 0.5156
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.4956 - acc: 0.9800 - val_loss: 0.7072 - val_acc: 0.5057
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 0.2814 - acc: 0.9850 - val_loss: 0.7029 - val_acc: 0.5210
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 0.1150 - acc: 1.0000 - val_loss: 0.7193 - val_acc: 0.5169
Epoch 5/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0513 - acc: 1.0000 - val_loss: 0.7168 - val_acc: 0.5294
Epoch 6/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0258 - acc: 1.0000 - val_loss: 0.7286 - val_acc: 0.5256
Epoch 7/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0139 - acc: 1.0000 - val_loss: 0.7234 - val_acc: 0.5344
Epoch 8/10
200/200 [========